In [1]:
import hashlib
list_cmt0 =['a952b726db5699f31090d9255c516514a11cad9af4bd0f3cd285d2722a55ca7d', 'd106da1c399218e5b18733d8a88bb5b0485b87d647af59320a73bff4514bd181', 'c099824f0600676bc8f32bcb764252b92a0e3c7780a9eee910ab95f4ddab05c5', '7e49dc5bae1af68df98f542f6752e764ef7b820e1d75e014f22b56b695841d6c', '902e9d9d5f70dd2a9b8a7a332ae57a6125d2112e1ccf93d9d57b3c00b03ee445', '2bf2e58a507580b1094a54626600307f1c85c1e7bea1be272141db022d91b1ec', 'f16ca6a7e263e275ec1ee0db59ae1490e23f15c17532caf0f60b86bf5e1e8d13', 'da73600c2ee2bace2aff6d22aaab0e02f2a78bb811ccebb4bc37bcc6729ef267', '802278be2ac0856fb95632d9389d3a241cb1489a6f21360be50b0158b94d7c74', '31aa3cdb91b1849d7ac58c09c454d5d23c6d94606e7831ab70694589f274c462', 'a72d5c9f0b2132b8276257eb139dafd91832d481af0f523127cd70be779b8638', 'f7cc8b27d4646666c25e67886bc9488f25348c46c6a3e6ed5d3e60f72e4b5e67', '71a5d0b1efe9efe7c53391843e9cddc1ee3420550058e96b2eb11cafa8c28498', '07db2db5a4e17a2bdd42014c0373af5aebd8a816fc6ad10dc064a7ea94993e77', '235c640907c3ebe4a43048c9c470fc0f35f2a2b0677e43e5af49c6c91e608934', '23673210206eb725ea4f6c47d1ed9301426b1a10e2f18a014ec997dd18ef75d3']
print(len(list_cmt0))
print(list_cmt0[4])
print(list_cmt0[10])
def hexhash(input):
    return hashlib.sha256(input.encode()).hexdigest()

16
902e9d9d5f70dd2a9b8a7a332ae57a6125d2112e1ccf93d9d57b3c00b03ee445
a72d5c9f0b2132b8276257eb139dafd91832d481af0f523127cd70be779b8638


In [2]:
#MERKLE TREE

class MerkleTree:
    def __init__(self, n_leaves):
        if(n_leaves%2==0):
            self.n_leaves = n_leaves
            self.depth = log(n_leaves, 2) 
            self.tree =  None
            self.merkle_proof =  None
            self.full = False
        else:
            raise Exception('Il numero delle foglie deve essere una porenza di 2')

    #METODI PUBBLICI 
    def make_tree(self, leaves):
        if isinstance(leaves, list) and len(leaves) == self.n_leaves and not(self.full): 
            self.tree = []
            self.tree.append(leaves)  
            self.__make_tree(leaves, self.depth-1) 
            self.full = True
        else:
            raise Exception('Il numero delle foglie deve essere pari a {}'.format(self.n_leaves))
       
    def get_root(self):
        if self.full:
            return self.tree[0][0]
        else:
            raise Exception('Albero non creato')

    def get_merkle_proof(self):
        if self.full and self.merkle_proof is not None:
            return self.merkle_proof

    def compute_proof(self, list_nodes):
        if all(self.full or 1 <= node <= 2**self.depth for node in list_nodes):
            self.merkle_proof={}
            list_leaves = sorted(list(map(lambda x: x+1 if x % 2 == 0 else x-1, list_nodes))) 
            self.__compute_merkle_proof(self.depth, list_leaves)
            list_leaves = list(filter(lambda x: x not in list_nodes, list_leaves)) 
            self.merkle_proof[self.depth] = dict(map(lambda x: (x, self.tree[self.depth][x]), list_leaves)) 
            #self.merkle_proof = sorted(self.merkle_proof.items())
            self.merkle_proof = {k: self.merkle_proof[k] for k in sorted(self.merkle_proof, key=lambda x: int(x))}
        else:
            raise Exception('Albero non creato o foglia non valida')

    def recompute_root(self, merkle_proof, leaves):
        if len(leaves)<self.n_leaves:
            return self.__recompute_root(merkle_proof, leaves)
    
    #METODI PRIVATI

    def __make_tree(self, list_nodes, depth): 
        if(depth>=0): 
            list_parent = self.__create_new_level(list_nodes) 
            self.tree.insert(0, list_parent)
            self.__make_tree(list_parent, depth-1) 

    def __create_new_level(self, child_nodes):
        parents = [] 
        for i in range(0, len(child_nodes), 2): 
            new_node = self.__compute_father_value(child_nodes[i], child_nodes[i+1]) 
            parents.append(new_node)
        return parents
        
    def __compute_father_value(self, sx, dx):
        return hexhash(sx+dx)
    
    def __compute_merkle_proof(self, depth, list_index):
        if(depth > 1): 
            list_parent = sorted(list(set(map(lambda elem: ((elem // 2) + 1) if (elem // 2) % 2 == 0 else (elem // 2) - 1, list_index))))  
            parent_proof = list(filter(lambda x: x*2 not in list_index and (x*2)+1 not in list_index, list_parent))
            if len(parent_proof)>0:
                self.merkle_proof[depth-1] = dict(map(lambda x: (x, self.tree[depth-1][x]), parent_proof)) 
            self.__compute_merkle_proof(depth-1, list_parent) 

    def __recompute_root(self, merkle_proof, nodes):
        if len(merkle_proof)==0:
            print("0",nodes[0])
            print("1",nodes[1])
            return self.__compute_father_value(nodes[0],nodes[1])
        else:
            index = max(merkle_proof.keys())
            current_level = merkle_proof[index]
            merged_nodes = {**current_level, **nodes}
            merged_nodes = dict(sorted(merged_nodes.items()))
            new_nodes = {}
            for key in merged_nodes:
                new_key = key//2
                if new_key in new_nodes.keys():
                    new_nodes[new_key] = self.__compute_father_value(new_nodes[new_key],merged_nodes[key])
                else:
                    new_nodes[new_key] = merged_nodes[key]
            del merkle_proof[index]
            return self.__recompute_root(merkle_proof, new_nodes)

In [4]:
t = 16
merkle_tree = MerkleTree(t)
merkle_tree.make_tree(list_cmt0)
c0 = merkle_tree.get_root()
#index_0 = [1, 2, 3, 5, 8, 9, 11, 12, 13]
index_0 = [0,2,5,7,10,12,13,15]
merkle_tree.compute_proof(index_0)
for i in merkle_tree.tree:
    print(i)
root= merkle_tree.get_root()
proof = merkle_tree.get_merkle_proof()
for x in proof:
    print(x, list(proof[x].keys()))
#list_cmt={4:"902e9d9d5f70dd2a9b8a7a332ae57a6125d2112e1ccf93d9d57b3c00b03ee445", 10:"a72d5c9f0b2132b8276257eb139dafd91832d481af0f523127cd70be779b8638"}
#recomp_root=merkle_tree.recompute_root(proof, list_cmt)

#print(c0==recomp_root)


['1204f87e5cc22b9a3327275ce99886deb8795607dcc6e17fafbd930d31c4c9b6']
['8d9f472a94e46f2db3f6125ba472fd987b696a0f03ada1a8ddda347faf3ce706', '1b31b1f20042f483e5af5e656c6f2025d48bf1c9e21dcf03de5393913e23130d']
['d505d64c00d13777616bbbd1cbe7fcb52d456d1cee74d7965851114332eb65ec', '7f1db0e3cfb6cce618ed55d55bb157dad1a5f9242428e1d6d79de671a2577bea', 'e5f045869d7df9b459e8f796df3aeb27e797911d53c0381155bc2c80d555b944', 'e9feedecf6cf4562747c1c718fb62c673e33de3ef46fef4471263cc38573e3c3']
['6bb73161cb9510c8cdc1fcca64f040a4df13a3d60ae3ae04cccec2cf6217f456', 'b97481329c041b4e6f3d504f83b0c6314275e3c1b556769f81cfca414ee62193', '97cae9d050becd6482478ec4fb3254ae7fda3363b45b441cc0eafe8ffeb75e5c', 'cb415b33f12f4d94781cab97262f60883a60fa015537ae6bed7746b2d581abf2', 'f85dd0ac7387e6a6745aaea6592b40b2ca886d4fdc11541c61219551cb947f80', '5719dcfeb5e76275999622c891e44b54521506a4d26c1764cf5e148ac754bd8f', '29d2505c4b656e866f405c0c26eebf485e6b9c1d3383f31ae6f51ac3a880fbcd', '1cbb0057a4888882a62b2815ea7a91137a9a304821f